In [15]:
# !pip install langchain==0.1.14 langchain-community==0.0.31 langchain-openai==0.1.2 langchain-chroma==0.1.1

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os

In [4]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Semantic_Book_Recommender')

In [6]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')

In [7]:
books['tagged_description']

,tagged_description
0,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982 A new 'Christie for Christmas' -...
2,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897 Lewis' work on the nature of lov...
4,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...
5192,9788172235222 On A Train Journey Home To North...
5193,9788173031014 This book tells the tale of a ma...
5194,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535 This collection of the timeless ...


In [8]:
raw_documents = TextLoader('tagged_description.txt', encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator='\n')
documents = text_splitter.split_documents(raw_documents)

Streaming output truncated to the last 5000 lines.


In [9]:
db_books = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

k argument is for the number of recommendations needed

In [10]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.', metadata={'source': 'tagged_description.txt'}),
 Document(page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects.", metadata={'source': 'tagged_description.txt'}),
 Document(page_content="9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a ra

In [11]:
books[books['isbn13']== int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [12]:
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
  recs = db_books.similarity_search(query, k= 50)
  books_list = []
  for i in range(0, len(recs)):
    books_list += [int(recs[i].page_content.strip('"').split()[0])]

  return books[books['isbn13'].isin(books_list)].head(top_k)

In [14]:
retrieve_semantic_recommendations('A book to teach children about animals')

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
85,9780030547744,0030547741,Where the Red Fern Grows,Wilson Rawls,Juvenile Fiction,http://books.google.com/books/content?id=IHpRw...,A young boy living in the Ozarks achieves his ...,2000.0,4.37,288.0,95.0,Where the Red Fern Grows: The Story of Two Dog...,9780030547744 A young boy living in the Ozarks...
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980 Even the youngest child can enjo...
707,9780140568196,0140568190,The Giraffe and the Pelly and Me,Roald Dahl;Quentin Blake,Candy,http://books.google.com/books/content?id=J7FdI...,"A Dahl story in which the giraffe, the pelican...",2001.0,3.81,32.0,16265.0,The Giraffe and the Pelly and Me,9780140568196 A Dahl story in which the giraff...
774,9780141309408,0141309407,Serena and the Sea Serpent,Garth Nix,Sea monsters,http://books.google.com/books/content?id=jb4vP...,Novel in the Aussie Bites series for early pri...,2000.0,3.59,79.0,33.0,Serena and the Sea Serpent,9780141309408 Novel in the Aussie Bites series...
810,9780142300848,0142300845,"Oliver and Albert, Friends Forever",Jean Van Leeuwen,Juvenile Fiction,http://books.google.com/books/content?id=-EBFs...,"Oliver makes friends with Albert, the new boy ...",2002.0,3.62,48.0,22.0,"Oliver and Albert, Friends Forever",9780142300848 Oliver makes friends with Albert...
812,9780142302279,0142302279,Dirty Beasts,Roald Dahl,Juvenile Nonfiction,NaN,Poems tell the stories of a smart pig who outw...,2002.0,4.02,32.0,3953.0,Dirty Beasts,9780142302279 Poems tell the stories of a smar...
815,9780142402498,0142402494,Pippi Longstocking,Astrid Lindgren,Juvenile Fiction,http://books.google.com/books/content?id=STE5P...,Relates the antics of a rambunctious girl who ...,2005.0,4.12,160.0,140107.0,Pippi Longstocking,9780142402498 Relates the antics of a rambunct...
886,9780152758516,0152758518,The Sleeping Giant and Other Stories,Eleanor Estes,Folklore,NaN,Three short tall tales : The sleeping giant --...,1948.0,4.00,101.0,10.0,The Sleeping Giant and Other Stories,9780152758516 Three short tall tales : The sle...
1078,9780241003008,0241003008,The Very Hungry Caterpillar,Eric Carle,Babytime resource,http://books.google.com/books/content?id=DpGEQ...,Eric Carle's children's classic is the story o...,1994.0,4.29,26.0,340101.0,The Very Hungry Caterpillar,9780241003008 Eric Carle's children's classic ...
1080,9780241912720,0241912725,Stuart Little,Elwyn Brooks White,"Little, Stuart (Fictitious character)",NaN,The adventures of the debonair mouse Stuart Li...,1946.0,3.89,144.0,109.0,Stuart Little,9780241912720 The adventures of the debonair m...
